### (간략) 구문 분석

1. spacy 구문분석

In [4]:
#pip install spacy
#!python -m spacy download ko_core_news_sm

In [5]:
import spacy

In [8]:
# 한국어 모델 로드 # "ko_core_news_sm"
nlp = spacy.load("ko_core_news_sm")

In [10]:
# 구문트리
doc = nlp("이것은 한국어로 된 문장입니다.")
doc

이것은 한국어로 된 문장입니다.

In [11]:
# 구문트리 출력하기 
# dep : 관계

for token in doc:
    print(f"token: {token.text} || token.dep_: {token.dep_} || token.head.text: {token.head.text}")

    

token: 이것은 || token.dep_: dislocated || token.head.text: 문장입니다
token: 한국어로 || token.dep_: nsubj || token.head.text: 된
token: 된 || token.dep_: ccomp || token.head.text: 문장입니다
token: 문장입니다 || token.dep_: ROOT || token.head.text: 문장입니다
token: . || token.dep_: punct || token.head.text: 문장입니다


In [12]:
# 도서지수가 높게 나왔을 때, 해당 이미지를 근거로 문장의 복잡성이 이렇기 때문에 지수가 높게 나왔다고 할 수 있음
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
# displacy 시각화

In [ ]:
import pandas as pd

In [15]:
# 데이터 가져와서 구문 분석
import pandas as pd
train_data = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t', encoding='utf-8')
train_data.to_csv('./data/ratings_train.csv', index=False, encoding='utf-8')
sample_data =train_data.iloc[100]['document']
sample_data

'신카이 마코토의 작화와,미유와 하나카나가 연기를 잘해줘서 더대박이였다.'

In [16]:
# 출력
doc = nlp(sample_data)
doc

신카이 마코토의 작화와,미유와 하나카나가 연기를 잘해줘서 더대박이였다.

In [18]:
for token in doc:
    print(f"token: {token.text} || token.dep_: {token.dep_} || token.head.text: {token.head.text}")

token: 신카이 || token.dep_: amod || token.head.text: 마코토의
token: 마코토의 || token.dep_: dislocated || token.head.text: 더대박이였다
token: 작화와 || token.dep_: nmod || token.head.text: 연기를
token: , || token.dep_: punct || token.head.text: 작화와
token: 미유와 || token.dep_: conj || token.head.text: 작화와
token: 하나카나가 || token.dep_: conj || token.head.text: 작화와
token: 연기를 || token.dep_: obj || token.head.text: 잘해줘서
token: 잘해줘서 || token.dep_: ROOT || token.head.text: 잘해줘서
token: 더대박이였다 || token.dep_: conj || token.head.text: 잘해줘서
token: . || token.dep_: punct || token.head.text: 더대박이였다


In [19]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

2. NLTK 구문분석

In [20]:
import konlpy
import nltk
from ckonlpy.tag import Twitter

In [21]:
# 형태소 분석기
okt = Twitter()

C:\Users\chunjae\AppData\Roaming\Python\Python311\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [22]:
# 사용자 사전 추가
okt.add_dictionary("미유와", "Noun")
okt.add_dictionary("하나카나", "Noun")

In [23]:
# twitter 형태소 분석
words = okt.pos(sample_data)
words

[('신카이', 'Noun'),
 ('마코토', 'Noun'),
 ('의', 'Josa'),
 ('작화', 'Noun'),
 ('와', 'Josa'),
 (',', 'Punctuation'),
 ('미유와', 'Noun'),
 ('하나카나', 'Noun'),
 ('가', 'Josa'),
 ('연기', 'Noun'),
 ('를', 'Josa'),
 ('잘', 'VerbPrefix'),
 ('해줘서', 'Verb'),
 ('더', 'Noun'),
 ('대박', 'Noun'),
 ('이', 'Josa'),
 ('였다', 'Verb'),
 ('.', 'Punctuation')]

In [26]:
# 문법을 지정해서 '구'형태로 묶기
# {<N.*>} 명사는 N이고 그 뒤에 임의의 단어를 붙일 수 있다.
# 명사구, 동사구, 형용사구 지정

grammar = """
NP: {<N.*>*<Suffix>?} # Noun phrase
VP: {<V.*>*} # Verb phrase
AP: {<A.*>*} # Adjective phrase
"""

parser = nltk.RegexpParser(grammar=grammar)
chunks =parser.parse(words)
print(chunks.pprint())

(S
  (NP 신카이/Noun 마코토/Noun)
  의/Josa
  (NP 작화/Noun)
  와/Josa
  ,/Punctuation
  (NP 미유와/Noun 하나카나/Noun)
  가/Josa
  (NP 연기/Noun)
  를/Josa
  (VP 잘/VerbPrefix 해줘서/Verb)
  (NP 더/Noun 대박/Noun)
  이/Josa
  (VP 였다/Verb)
  ./Punctuation)
None


In [27]:
# 시각화
# 특정 명사가 중복 됐을 때, 고유명사로 보는 경우가 있음
# 신카이, 마코토를 신카이마코토 복합명사로 묶음
chunks.draw()